<a href="https://colab.research.google.com/github/seoyen1122/solar_rag/blob/main/mmlu_pro/psychology.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install -q openai langchain tiktoken faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 118.3 MB/s eta 0:00:00


In [2]:
!pip install -U langchain-upstage langchain-community pypdf

INFO: pip is looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 57.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 105.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.4 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.22.1
    Uninstalling tokenizers-0.22.1:
      Successfully uninstalled tokenizers-0.22.1
ERROR: pip's dependency resolver does no

In [3]:
UPSTAGE_API_KEY = "up_g7T2cQoLKZH6Oi2n4MHOW706XAdSs"

In [4]:
import os
os.environ["UPSTAGE_API_KEY"] = UPSTAGE_API_KEY

In [5]:
import re
from pathlib import Path

from pypdf import PdfReader
from langchain.schema import Document
from langchain_community.vectorstores import FAISS
from langchain_upstage import UpstageEmbeddings

In [6]:
CHAPTERS = [
    {"num": 1, "title": "Introduction to Psychology", "start": 7,  "end": 34},
    {"num": 2, "title": "Psychological Research",     "start": 35, "end": 70},
    {"num": 3, "title": "Biopsychology",              "start": 71, "end": 108},
    {"num": 4, "title": "States of Consciousness",    "start": 109, "end": 143},
    {"num": 5, "title": "Sensation and Perception",    "start": 145, "end": 179},
    {"num": 6, "title": "Learning",    "start": 181, "end": 211},
    {"num": 7, "title": "Thinking and Intelligence",    "start": 213, "end": 246},
    {"num": 8, "title": "Memory",    "start": 247, "end": 277},
    {"num": 9, "title": "Livespan Development",    "start": 279, "end": 320},
    {"num": 10, "title": "Emotion and Motivation",    "start": 321, "end": 357},
    {"num": 11, "title": "Personality",    "start": 359, "end": 397},
    {"num": 12, "title": "Social Psychology",    "start": 399, "end": 444},
    {"num": 13, "title": "Industrial-Organizational Psychology",    "start": 447, "end": 484},
    {"num": 14, "title": "Stress, Lifestyle, and Health",    "start": 485, "end": 535},
    {"num": 15, "title": "Psychological Disorders",    "start": 537, "end": 598},
    {"num": 16, "title": "Therapy and Treatment",    "start": 599, "end": 733},
]

In [8]:
cd /content/drive/MyDrive/Colab Notebooks/2025_2/nlp

/content/drive/MyDrive/Colab Notebooks/2025_2/nlp


In [7]:
pip install jsonlines

In [9]:
import re
from pathlib import Path

import jsonlines
from pypdf import PdfReader

In [10]:
PDF_PATH = "psychology2e.pdf"
OUTPUT_JSONL = "entries_psychology.jsonl"
BASE_URL = "https://openstax.org/books/psychology-2e"

In [11]:
def extract_pages_text(reader: PdfReader, start_page: int, end_page: int) -> str:
    texts = []
    for i in range(start_page, end_page):
        if i < 0 or i >= len(reader.pages):
            continue
        try:
            t = reader.pages[i].extract_text() or ""
        except Exception as e:
            print(f"[WARN] page {i} extract_text 실패: {e}")
            t = ""
        if t.strip():
            texts.append(t)
    return "\n".join(texts)

In [12]:
def normalize_whitespace(text: str) -> str:
    text = text.replace("\t", " ")
    text = "\n".join(line.rstrip() for line in text.splitlines())
    # 연속 공백 줄이기
    text = re.sub(r"[ ]{2,}", " ", text)
    return text

In [13]:
def split_chapter_into_sections(ch_num: int, chapter_text: str):
    """
    한 챕터 텍스트를 섹션(1.1, 1.2, ..., Key Terms, Summary, Review...) 단위로 나눔.

    반환: [{"section_title": "...", "text": "..."}, ...]
    """
    # 줄 단위로 먼저 나눔
    text = normalize_whitespace(chapter_text)
    lines = text.splitlines()

    # 섹션 헤더 패턴: "1.1 What Is Psychology?" 같이 챕터번호로 시작하는 제목
    # + 요약/연습문제 섹션들 (필요에 따라 추가)
    header_regex = re.compile(
        rf"^(?:{ch_num}\.\d+\s+.+|Key Terms|Summary|Review Questions|Critical Thinking Questions|Personal Application Questions)\s*$"
    )

    # 헤더 위치 찾기
    headers = []
    for idx, line in enumerate(lines):
        if header_regex.match(line.strip()):
            headers.append((idx, line.strip()))

    if not headers:
        # 헤더 못 찾으면 챕터 전체를 하나의 섹션으로 반환
        print(f"[WARN] chapter {ch_num} 에서 섹션 헤더를 찾지 못했습니다. 전체를 1개로 처리.")
        return [{
            "section_title": f"Chapter {ch_num} (full)",
            "text": text,
        }]

    sections = []
    for i, (line_idx, header_text) in enumerate(headers):
        start_line = line_idx + 1  # 헤더 다음 줄부터 내용 시작
        end_line = headers[i + 1][0] if i + 1 < len(headers) else len(lines)

        body_lines = lines[start_line:end_line]
        body = "\n".join(body_lines).strip()

        # body가 너무 짧으면 헤더 뒤 내용이 없는 경우 → 그냥 스킵할 수도 있음
        if not body:
            continue

        sections.append({
            "section_title": header_text,
            "text": body,
        })

    return sections

In [17]:
def main():
    pdf_path = Path(PDF_PATH)
    assert pdf_path.exists(), f"PDF가 없습니다: {pdf_path}"

    reader = PdfReader(str(pdf_path))

    with jsonlines.open(OUTPUT_JSONL, mode="w") as writer:
        for ch in CHAPTERS:
            ch_num = ch["num"]
            ch_title = ch["title"]
            start = ch["start"]
            end = ch["end"]

            print(f"[CHAPTER {ch_num}] {ch_title} (pages {start}~{end})")

            chapter_text = extract_pages_text(reader, start, end)
            sections = split_chapter_into_sections(ch_num, chapter_text)
            print(f"  - 섹션 개수: {len(sections)}")

            entry = {
                "source_url": f"{BASE_URL}#chapter-{ch_num}",
                "title": f"Chapter {ch_num} {ch_title}",
                "chunk_list": sections,   # [{section_title, text}, ...]
            }

            writer.write(entry)

    print(f"[완료] {OUTPUT_JSONL} 에 chapter/section 단위 entries 저장 완료.")


if __name__ == "__main__":
    main()

[CHAPTER 1] Introduction to Psychology (pages 7~34)
  - 섹션 개수: 6
[CHAPTER 2] Psychological Research (pages 35~70)
  - 섹션 개수: 11
[CHAPTER 3] Biopsychology (pages 71~108)
  - 섹션 개수: 11
[CHAPTER 4] States of Consciousness (pages 109~143)
  - 섹션 개수: 12
[CHAPTER 5] Sensation and Perception (pages 145~179)
  - 섹션 개수: 12
[CHAPTER 6] Learning (pages 181~211)
  - 섹션 개수: 10
[CHAPTER 7] Thinking and Intelligence (pages 213~246)
  - 섹션 개수: 6
[CHAPTER 8] Memory (pages 247~277)
  - 섹션 개수: 9
[CHAPTER 9] Livespan Development (pages 279~320)
  - 섹션 개수: 12
[CHAPTER 10] Emotion and Motivation (pages 321~357)
  - 섹션 개수: 12
[CHAPTER 11] Personality (pages 359~397)
  - 섹션 개수: 18
[CHAPTER 12] Social Psychology (pages 399~444)
  - 섹션 개수: 13
[CHAPTER 13] Industrial-Organizational Psychology (pages 447~484)
  - 섹션 개수: 13
[CHAPTER 14] Stress, Lifestyle, and Health (pages 485~535)
  - 섹션 개수: 13
[CHAPTER 15] Psychological Disorders (pages 537~598)
  - 섹션 개수: 18
[CHAPTER 16] Therapy and Treatment (pages 599~733)
  

In [20]:
upstage_embeddings = UpstageEmbeddings(api_key=UPSTAGE_API_KEY, model="embedding-passage")

In [24]:
#!/usr/bin/env python3
"""
create_faiss_index.py

- 'entries.jsonl' (Semantic Chunking된 파일)을 읽어옵니다.
- 각 청크(섹션)를 로드합니다.
- (안전 장치) 만약 섹션 텍스트가 1000자를 넘으면, 1000자 단위로 더 잘게 자릅니다.
- 이 과정에서 'title', 'source_url', 'section_title' 메타데이터를 모두 보존합니다.
- BGE-small 임베딩 모델을 사용하여 모든 청크를 임베딩합니다.
- 'faiss_index_philosophy'라는 이름의 로컬 FAISS 인덱스로 저장합니다.
"""

import jsonlines
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.docstore.document import Document
import sys
import time

# ----------------------------------------------------
# 1. 설정값
# ----------------------------------------------------
JSONL_FILE = "entries_psychology.jsonl"             # 입력 파일 (스크래핑 결과)
INDEX_NAME = "faiss_index_psychology"    # 저장할 FAISS 인덱스 이름

# "Safety Net" 청킹 설정 (H2 섹션이 너무 클 경우 대비)
CHUNK_SIZE = 1000   # 청크 최대 글자 수
CHUNK_OVERLAP = 100 # 청크 겹침

# ----------------------------------------------------
# 2. 임베딩 모델 로드 (BGE-m3)
# ----------------------------------------------------
model_name = "BAAI/bge-m3"
model_kwargs = {'device': 'cuda'} # Colab GPU 사용
encode_kwargs = {'normalize_embeddings': True} # BGE 모델 권장 사항

try:
    upstage_embeddings
except Exception as e:
    print(f"Error loading embedding model. Do you have a GPU runtime? {e}")
    # (GPU 런타임이 아닐 경우 CPU로 fallback)
    # model_kwargs = {'device': 'cpu'}
    # embedding_model = HuggingFaceEmbeddings(
    #     model_name=model_name,
    #     model_kwargs=model_kwargs,
    #     encode_kwargs=encode_kwargs
    # )
print("Embedding model loaded.")

# ----------------------------------------------------
# 3. JSONL 로드 및 '안전 장치' 청킹
# ----------------------------------------------------
print(f"Loading '{JSONL_FILE}' and applying safety net chunking...")

# '안전 장치' (H2 섹션이 너무 클 경우를 대비한) 텍스트 분할기
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=CHUNK_OVERLAP,
    length_function=len,
)

all_final_chunks = [] # 최종적으로 FAISS에 들어갈 Document 객체 리스트

try:
    with jsonlines.open(JSONL_FILE, 'r') as reader:
        for entry in reader:
            # (1) 기본 메타데이터 (페이지 레벨)
            base_metadata = {
                "source": entry.get("source_url", "N/A"),
                "title": entry.get("title", "N/A"),
            }

            # (2) Semantic Chunking된 'chunk_list' 순회
            for chunk in entry.get("chunk_list", []):
                section_text = chunk.get("text")
                section_title = chunk.get("section_title", "N/A")

                if not section_text:
                    continue

                # (3) H2 섹션 텍스트가 CHUNK_SIZE(1000자)를 넘을 경우,
                #     text_splitter가 이 텍스트를 더 작은 '미니 청크'로 자름
                split_texts = text_splitter.split_text(section_text)

                # (4) 이 '미니 청크'들을 Document 객체로 변환
                for text_piece in split_texts:
                    # 메타데이터에 'section' 정보를 추가
                    final_metadata = base_metadata.copy()
                    final_metadata["section"] = section_title

                    new_doc = Document(page_content=text_piece, metadata=final_metadata)
                    all_final_chunks.append(new_doc)

except FileNotFoundError:
    print(f"Error: '{JSONL_FILE}' not found. Please run the scraping script first.")
    sys.exit()

print(f"Total 'mini-chunks' to be indexed: {len(all_final_chunks)}")

# ----------------------------------------------------
# 4. FAISS 임베딩 및 저장
# ----------------------------------------------------
if all_final_chunks:
    print("Starting FAISS index creation... (This may take a long time)")
    start_time = time.time()

    # FAISS.from_documents()를 사용하면
    # 텍스트 청크는 임베딩되고, 메타데이터는 그대로 벡터 스토어에 저장됩니다.
    db_philosophy = FAISS.from_documents(all_final_chunks, upstage_embeddings)

    end_time = time.time()
    print(f"FAISS index created successfully in {end_time - start_time:.2f} seconds.")

    # 생성된 인덱스를 파일로 저장
    db_philosophy.save_local(INDEX_NAME)

    print(f"FAISS index saved to folder: '{INDEX_NAME}'")
else:
    print("No chunks were created. FAISS index not built.")

Embedding model loaded.
Loading 'entries_psychology.jsonl' and applying safety net chunking...
Total 'mini-chunks' to be indexed: 2232
Starting FAISS index creation... (This may take a long time)
FAISS index created successfully in 330.44 seconds.
FAISS index saved to folder: 'faiss_index_psychology'
